In [20]:
import flowermd
import gsd
import gsd.hoomd
import hoomd
import matplotlib.pyplot as plt
import mbuild as mb
import numpy as np
import warnings
from cmeutils.visualize import FresnelGSD
from flowermd.base import Pack, Simulation, System, Molecule
from flowermd.base.forcefield import BaseHOOMDForcefield
from flowermd.library import KremerGrestBeadSpring, LJChain
from flowermd.library.forcefields import BeadSpring
from flowermd.utils import get_target_box_number_density
from mbuild.compound import Compound
from mbuild.lattice import Lattice
warnings.filterwarnings('ignore')

class Graphene(System):
    def __init__(
        self,
        x_repeat,
        y_repeat,
        n_layers,
        base_units=dict(),
        periodicity=(True, True, False),
    ):
        surface = mb.Compound(periodicity=periodicity)
        a = 3**.5
        lattice = Lattice(
            lattice_spacing=[a,a,a],
            lattice_vectors=  [[a,0,0],[a/2,3/2,0],[0,0,1]],
            lattice_points={"A": [[1/3,1/3,0], [2/3, 2/3, 0]]},
        ) 
        Flakium = Compound(name="F", element="F") # defines a carbon atom that will be used to populate lattice points
        layers = lattice.populate(
            compound_dict={"A": Flakium}, x=x_repeat, y=y_repeat, z=n_layers
        ) # populates the lattice using the previously defined carbon atom for every "A" site, repeated in all x,y, and z directions
        surface.add(layers) # adds populated carbon lattice layers to the 'surface' compound, which represents our graphene structure 
        surface.freud_generate_bonds("F", "F", dmin=0.9, dmax=1.1) # generates bonds depending on input distance range, scales with lattice
        surface_mol = Molecule(num_mols=1, compound=surface) # wraps into a Molecule object, creating "1" instance of this molecule

        super(Graphene, self).__init__(
            molecules=[surface_mol],
            base_units=base_units,
        )

    def _build_system(self):
        return self.all_molecules[0]
#OK, so we want to initialize a system with some chains and some flakes
kg_chain = LJChain(lengths=10,num_mols=500)
sheet = Graphene(x_repeat=5, y_repeat=5, n_layers=1, periodicity=(False, False, False))
system = Pack(molecules=[Molecule(compound=sheet.all_molecules[0], num_mols=2), kg_chain], density=0.01, packing_expand_factor = 6, seed=2, overlap=1.1)
system.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
# this FF is for research question
# WCA = 2 **1/6, kT = 3.0, needs thousand chains, maybe 10-12 flakes. large system, need lots of steps. 5e6 probably good. 
ff = BeadSpring(
    #r_cut=2.5, 
    r_cut=2**(1/6),
    beads={
        "A": dict(epsilon=1.0, sigma=1.0),  # chains
        "F": dict(epsilon=1.0, sigma=1.0),  # flakes
    },
    bonds={
        "F-F": dict(r0=1.0, k=1000),
        "A-A": dict(r0=1.0, k=1000.0),  # increased k to avoid chain collapse
    },
    angles={
        "A-A-A": dict(t0=2* np.pi / 3., k=100.0),   # moderate stiffness for chains
        "F-F-F": dict(t0=2 * np.pi / 3., k=5000),
    },
    dihedrals={
        "A-A-A-A": dict(phi0=0.0, k=0, d=-1, n=2), # need to turn this on later, messed up with straight chains
        "F-F-F-F": dict(phi0=0.0, k=500, d=-1, n=2),
    }
)
gsd ="500_10mers_2f_kT3.gsd"
log ="500_10mers_2f_kT3.txt"
cpu = hoomd.device.CPU()
sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=ff.hoomd_forces, device=cpu, dt = 0.0005, gsd_write_freq=int(1000), log_file_name = log, gsd_file_name = gsd)
sim.run_NVT(n_steps=1e6, kT=3, tau_kt=.1)
sim.flush_writers()
sim_visualizer = FresnelGSD(gsd_file=gsd, frame=-1, view_axis=(1, 1, 1))
sim_visualizer.view()

*Warning*: dihedral.harmonic: specified K <= 0


Initializing simulation state from a gsd.hoomd.Frame.
Step 1000 of 1000000; TPS: 1042.72; ETA: 16.0 minutes
Step 2000 of 1000000; TPS: 1153.19; ETA: 14.4 minutes
Step 3000 of 1000000; TPS: 1188.68; ETA: 14.0 minutes
Step 4000 of 1000000; TPS: 1209.43; ETA: 13.7 minutes
Step 5000 of 1000000; TPS: 1218.68; ETA: 13.6 minutes
Step 6000 of 1000000; TPS: 1227.92; ETA: 13.5 minutes
Step 7000 of 1000000; TPS: 1232.52; ETA: 13.4 minutes
Step 8000 of 1000000; TPS: 1238.45; ETA: 13.4 minutes
Step 9000 of 1000000; TPS: 1243.13; ETA: 13.3 minutes
Step 10000 of 1000000; TPS: 1243.45; ETA: 13.3 minutes
Step 11000 of 1000000; TPS: 1245.79; ETA: 13.2 minutes
Step 12000 of 1000000; TPS: 1245.23; ETA: 13.2 minutes
Step 13000 of 1000000; TPS: 1248.37; ETA: 13.2 minutes
Step 14000 of 1000000; TPS: 1250.01; ETA: 13.1 minutes
Step 15000 of 1000000; TPS: 1250.9; ETA: 13.1 minutes
Step 16000 of 1000000; TPS: 1252.73; ETA: 13.1 minutes
Step 17000 of 1000000; TPS: 1253.48; ETA: 13.1 minutes
Step 18000 of 1000000

In [ ]:
# for 2 chains, flake # does not matter at all, was able to go up to 12 flakes with ease
# 22 chains is the limit

In [ ]:
data = np.genfromtxt("500_10mers_2f_kT3.txt", names=True)
pe = data["mdcomputeThermodynamicQuantitiespotential_energy"]
x = data["flowermdbasesimulationSimulationtimestep"]
plt.xlabel("Timestep")
plt.pelabel("Potential Energy")

In [ ]:
plt.plot(x,pe)